# Timestamp on testnet - BX Exercise
In this exercise, we commit a set of data to an unspendable bitcoin output.

<hr style="border: 0.5px dashed #000;">

## 1. Restore wallet and generate spending key pairs.



In [ ]:
## my_mnemonic ="word0 word1 word2 ..."
# hd_master_private=$( )
# hd_master_public=$( )

my_mnemonic="seven mail crash you unit small assume express wedding cloud work potato"
hd_master_private=$(bx mnemonic-to-seed $my_mnemonic | bx hd-new)
hd_master_public=$(bx hd-to-public $hd_master_private)


We will be spending a p2pkh output script controlled by this key.

* `m/44'/1'/1'/0/4`

In [ ]:
# hd_m_44h_1h_1h_0_4=

# privatekey_44h_1h_1h_0_4=
# publickey_44h_1h_1h_0_4=
# publickeyhash_44h_1h_1h_0_4=
# address_44h_1h_1h_0_4=

hd_m_44h_1h_1h_0_4=$(bx hd-private --hard --index 44 $hd_master_private \
| bx hd-private --hard --index 1  \
| bx hd-private --hard --index 1  \
| bx hd-private --index 0 \
| bx hd-private --index 4)

privatekey_44h_1h_1h_0_4=$(bx hd-to-ec $hd_m_44h_1h_1h_0_4)
publickey_44h_1h_1h_0_4=$(bx hd-to-public $hd_m_44h_1h_1h_0_4 | bx hd-to-ec)
publickeyhash_44h_1h_1h_0_4=$(bx sha256 $publickey_44h_1h_1h_0_4 | bx ripemd160)
address_44h_1h_1h_0_4=$(bx hd-to-public $hd_m_44h_1h_1h_0_4 | bx hd-to-ec | bx ec-to-address --version 111)

echo $address_44h_1h_1h_0_4
echo $publickeyhash_44h_1h_1h_0_4

### Fetch previous UTXO's you are spending.

In [ ]:
# bx fetch-tx [transaction_hash] -f json | jq ".transaction.outputs[output_index]"

bx fetch-tx 11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea -f json | jq ".transaction.outputs[4]"

In [ ]:
# previous_txid=
# previous_output_index=
# previous_output_amount=

previous_txid=11a8587f5077fc7f69e3349797fd74f99864d27716d6b48ac51734cb3bec03ea
previous_output_index=4
previous_output_amount=499550

## 2. Sign input and output.

### A. Encode op_return output script.

* Commit arbitrary data to a merkle root.
* [Ascii to hex conversion tool.](https://www.asciitohex.com)

In [1]:
# the ups and downs of downing street.
# 0x746865 0x757073 0x616e64 0x646f776e73 0x6f6620 0x646f776e696e67 0x737472656574

data_leaf0=746865
data_leaf1=757073
data_leaf2=616e64
data_leaf3=646f776e73
data_leaf4=6f6620
data_leaf5=646f776e696e67
data_leaf6=737472656574

hash0=$(bx bitcoin160 $data_leaf0)
hash1=$(bx bitcoin160 $data_leaf1)
hash2=$(bx bitcoin160 $data_leaf2)
hash3=$(bx bitcoin160 $data_leaf3)
hash4=$(bx bitcoin160 $data_leaf4)
hash5=$(bx bitcoin160 $data_leaf5)
hash6=$(bx bitcoin160 $data_leaf6)

hash01=$(bx bitcoin160 $hash0$hash1)
hash23=$(bx bitcoin160 $hash2$hash3)
hash45=$(bx bitcoin160 $hash4$hash5)
hash66=$(bx bitcoin160 $hash6$hash6)

hash0123=$(bx bitcoin160 $hash01$hash23)
hash4566=$(bx bitcoin160 $hash45$hash66)

root=$(bx bitcoin160 $hash0123$hash4566)
echo $root


23316c9e53f6cec74c2331737e32e42ab02426e7


**Embed merkle root into op_return output script**

In [2]:
# output_script_return=

output_script_return=$(bx script-encode "return [23316c9e53f6cec74c2331737e32e42ab02426e7]")
echo $output_script_return

6a1423316c9e53f6cec74c2331737e32e42ab02426e7


### B. Encode p2pkh script for the change output.
* Please send testnet coins to p2pkh address `n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw`

In [ ]:
bx address-decode n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw

In [ ]:
# output_script_p2pkh=

output_script_p2pkh=$(bx script-encode "DUP HASH160 [e48199d47742b245464b1366d95ef26aa4c8bb2c] EQUALVERIFY CHECKSIG")
echo $output_script_p2pkh


### C. Compute amount for p2pkh output

In [ ]:
# tx bytes: 4 + 1 + 1*(32+4+1+1+72+1+33+4) + 1 + 1*(8+1+25) + 1*(8+1+1+20)+ 4 

# tx_byte_count=

tx_byte_count=$(expr 4 + 1 + 1 \* 148 + 1 + 1 \* 34 + 1 \* 30 + 4)
echo $tx_byte_count

In [ ]:
# P2PKH output amount.
# output_amount_p2pkh=

output_amount_p2pkh=$(expr $previous_output_amount - 1 \* 400 - $tx_byte_count \* 1)
echo $output_amount_p2pkh


### D. Construct transaction.

In [ ]:
# bx tx-encode \
# --input [previous tx id]:[index]:[sequence] \
# --output [output script]:[output amount]

my_tx=$(bx tx-encode \
--input $previous_txid:$previous_output_index:4294967295 \
--output $output_script_return:0 \
--output $output_script_p2pkh:$output_amount_p2pkh)



### E. Sign transaction & finalise transaction.

**Fetch previous output script.**

In [ ]:
# bx fetch-tx --format json [previous tx id] 

# previous_output_script=

previous_output_script=$(bx fetch-tx --format json $previous_txid \
| jq -r ".transaction.outputs[4].script")

echo $previous_output_script

**Sign transaction.**

In [ ]:
# bx input-sign [private key] "previous output script" [transaction template]

# signature=

signature=$(bx input-sign --sign_type all --index 0 $privatekey_44h_1h_1h_0_4 "$previous_output_script" $my_tx)


**Set input into finalised transaction**

In [ ]:
# bx input-set "[signature] [public key point]" [transaction template]

# my_tx=

my_tx=$(bx input-set --index 0 "[$signature] [$publickey_44h_1h_1h_0_4]" $my_tx)
echo $my_tx | bx tx-decode


In [ ]:
# Validate.
bx validate-tx $my_tx